In [3]:
import requests
import pandas as pd
#read website date into a dataframe
Toronto_data=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
Toronto_data=Toronto_data[0]

# stack all the data into a single column
Toronto_datanew = pd.Series(Toronto_data.values.ravel('F'))

Toronto_code = Toronto_datanew.to_frame()
Toronto_code.columns = ['A']

#Extract the PostalCode and Borough 
Toronto_code['PostalCode']=Toronto_code['A'].astype(str).str[0:3]
Toronto_code['Borough']=Toronto_code['A'].astype(str).str[3:]

# delete all records with Borough as 'Not Assigned'
Toronto_code = Toronto_code[Toronto_code.Borough != 'Not assigned']
Toronto_code = Toronto_code[['PostalCode','Borough']]

#Extract the data for Neighborhood 
import re
Toronto_code['Neighborhood'] = Toronto_code['Borough'].apply(lambda x: re.findall('\((.*?)\)',x))
df=Toronto_code['Borough'].str.split('(', expand=True)
df.columns = ['A','B','C']
Toronto_code['Borough'] = df['A'] 
Toronto_code = Toronto_code[['PostalCode','Borough','Neighborhood']]
Toronto_code['Neighborhood'] = Toronto_code['Neighborhood'].str.get(0)
#Sort the data by POstal Code and Borough
Toronto_code = Toronto_code.sort_values(by =['PostalCode','Borough'])

#Check for duplicate entrees for Postal Code
Toronto_code1 = Toronto_code[Toronto_code.Neighborhood != 'Not assigned']

#Total Records is equal to Total unique records for column PostalCode in Toronto_code1

# Merge the Neighborhood Data if PostalCode and Borough are Same

NumofRows = len(Toronto_code.index)
count = 0
nextcount = 1
while count!=NumofRows:
    if nextcount == NumofRows:
        break
    if ((Toronto_code['PostalCode'].iloc[count] == Toronto_code['PostalCode'].iloc[nextcount]) & (Toronto_code['PostalCode'].iloc[count] == Toronto_code['Borough'].iloc[nextcount])):
        Toronto_code['Neighborhood'].iloc[count] = (Toronto_code['PostalCode'].iloc[count] + ' , ' + Toronto_code['PostalCode'].iloc[nextcount])
        Toronto_code.drop(df.index[[nextcount]])
    count = count + 1
    nextcount = nextcount +1


In [4]:
Toronto_code.shape

(103, 3)

In [5]:
#extract the Longitute and latitude data
Toronto_geodata=pd.read_csv("http://cocl.us/Geospatial_data")

# Sort the data by Postal Code
Toronto_geodata = Toronto_geodata.sort_values(by =['Postal Code'])
Toronto_geodata = Toronto_geodata.rename(columns={"Postal Code": "PostalCode"})

# Add the longitude and latitude date to Original data frame
Toronto_code = Toronto_code.merge(Toronto_geodata, on="PostalCode", how = 'inner')


#### Final Dataframe of Toronto Zip Coes starting with M

In [50]:
Toronto_code

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


#### Getting Ready to Explore Toronto Neighborhood

In [67]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Create a map of Toronto with neighborhoods superimposed on top

In [56]:
# create map of Toronto using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_code['Latitude'], Toronto_code['Longitude'], Toronto_code['Borough'], Toronto_code['Neighborhood']):
    label = '{}, {}'.format(Toronto_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Cluster the Neighborhood

Run *k*-means to cluster the neighborhood into 5 clusters.

In [64]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering0 = Toronto_code.drop('Neighborhood', 1)
toronto_grouped_clustering1 = toronto_grouped_clustering0.drop('Borough', 1)
toronto_grouped_clustering2 = toronto_grouped_clustering1.drop('PostalCode', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering2)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

#End of Analysis

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2])